<a href="https://colab.research.google.com/github/turnbullfordsdt/Main-DSDT-Example-Repo/blob/main/Time_Series_Analysis_With_Imputation_%26_EXOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [269]:
from statsmodels.api import tsa

In [270]:
from statsmodels import api

In [271]:
# get time series data
data = api.datasets.get_rdataset('airquality').data

In [272]:
from sklearn.impute import KNNImputer

In [273]:
knn_imputer = KNNImputer(n_neighbors=5)
data_numpy = knn_imputer.fit_transform(data)

In [274]:
data[data.columns] = data_numpy

In [275]:
y = data['Temp']
X = data.drop(columns = 'Temp')
data

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67.0,5.0,1.0
1,36.0,118.0,8.0,72.0,5.0,2.0
2,12.0,149.0,12.6,74.0,5.0,3.0
3,18.0,313.0,11.5,62.0,5.0,4.0
4,18.2,159.0,14.3,56.0,5.0,5.0
...,...,...,...,...,...,...
148,30.0,193.0,6.9,70.0,9.0,26.0
149,31.0,145.0,13.2,77.0,9.0,27.0
150,14.0,191.0,14.3,75.0,9.0,28.0
151,18.0,131.0,8.0,76.0,9.0,29.0


In [276]:
import optuna
from numpy import array
from statsmodels.api import tsa
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

def objective(trial):
  # Suggest ARIMA parameters
  p = trial.suggest_int("p", 0, 5)
  d = trial.suggest_int("d", 0, 2)
  q = trial.suggest_int("q", 0, 5)

  cv_score_list = []

  tss = TimeSeriesSplit()

  split_set = tss.split(data)

  for train_indices, test_indices in split_set:

    x_train = X.iloc[train_indices]

    y_train = y.iloc[train_indices]

    model = tsa.ARIMA(y_train, x_train, order=(p, d, q))

    fit_model = model.fit()

    x_test = X.iloc[test_indices]

    y_test = y.iloc[test_indices]

    num_steps = len(y_test)

    y_pred = fit_model.forecast(steps = num_steps, exog = x_test)

    score = mean_squared_error(y_test, y_pred)

    cv_score_list.append(score)

  averaged_score = np.mean(cv_score_list)

  return averaged_score






In [277]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=3)

print("Best params:", study.best_params)
print("Best score:", study.best_value)

[I 2025-09-30 00:58:54,598] A new study created in memory with name: no-name-98b9765c-faa1-4fb0-8dba-9d931d50ac89
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
[I 2025-09-30 00:58:55,281] Trial 0 finished with value: 111.74683336790585 and parameters: {'p': 2, 'd': 1, 'q': 0}. Best is trial 0 with value: 111.74683336790585.
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/pyth

Best params: {'p': 2, 'd': 2, 'q': 4}
Best score: 94.76844452171278
